In [1]:
import openml
openml.config.apikey = 'eb60f811ca0d9f4f846d59be082919f9'

from tqdm import tqdm

import multiprocessing as mp

In [2]:
task_ids = [31, 10101, 3913, 3, 3917, 9957, 9946, 3918, 3903, 37, 9971, 9952, 3902, 49, 43, 9978, 10093, 219, 9976, 14965, 6, 9977, 53, 11, 15, 16, 14, 32, 3549, 12, 9981, 18, 28, 2074, 29, 45, 125922, 9960, 9964, 22, 2079,
            14969, 3560, 14952, 125920, 23, 3904, 3022, 9985, 9910, 14970, 3021, 3481, 7592, 3573, 146824, 146820, 146822, 146195, 146800, 146817, 146819, 146821, 167119, 14954, 167141, 167140, 167120, 167125, 146825, 167124, 167121]


In [3]:
def get_dataset_by_task_id(id):
    task = openml.tasks.get_task(id)
    X, y, categorical_indicator, attribute_names = task.get_dataset().get_data()
    return {
        'id': id, 'X': X, 'y': y,
        'categorical_indicator': categorical_indicator,
        'attribute_names': attribute_names
    }

pool = mp.Pool(mp.cpu_count())
datasets = list(tqdm(pool.imap(get_dataset_by_task_id, task_ids), total=len(task_ids)))

Process SpawnPoolWorker-1:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/queues.py", line 368, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'get_dataset_by_task_id' on <module '__main__' (built-in)>
Process SpawnPoolWorker-2:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.fr

KeyboardInterrupt: 

## Class imbalance

In [ ]:
class_imbalance_list = []

for ds in datasets:
    df = ds['X']
    last_col = df.columns[-1]
    
    uniques = df[last_col].unique()
    if len(uniques) == 2:
        perc = min(len(df[df[last_col] == u]) / len(df) for u in uniques)

        id = ds['id']
        class_imbalance_list.append({
            'id': id,
            'perc': perc,
            #'n_rows': len(df),
            #'n_cols': df.shape[1]
        })

In [ ]:
# print('(id, class_with_least_percentage_of_representation)')
# sorted(class_imbalance_dict.items(), key=lambda x: x[1])
class_imbalance_list = sorted(class_imbalance_list, key=lambda x: x['perc'])
for l in class_imbalance_list:
    print(l)

In [ ]:
from decisiontree import DecisionTree

from copy import deepcopy
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import openml
with open('apikey.txt', 'r') as f:
    openml.config.apikey = f.read()

In [ ]:
def get_result(dataset):
    try:
        id = dataset['id']
        task = openml.tasks.get_task(id)
        data, _, _, _ = task.get_dataset().get_data()
        data = data.dropna()

        X = data.values[:, :-1]
        y = data.values[:, -1]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=42)

        max_depth = int(np.sqrt(X.shape[1]))

        dt = DecisionTree(max_depth=max_depth, desimbalancer=False)
        dt.fit(X_train, y_train)
        res_false = dt.evaluate(X_test, y_test)

        dt = DecisionTree(max_depth=max_depth, desimbalancer=True)
        dt.fit(X_train, y_train)
        res_true = dt.evaluate(X_test, y_test)

        line = deepcopy(dataset)
        line['result_false'] = res_false
        line['result_true'] = res_true
        
        return line
    except Exception:
        return None
    
pool = mp.Pool(mp.cpu_count())
results = list(tqdm(pool.imap(get_result, class_imbalance_list[:20]), total=20))
    

In [ ]:
results = [r for r in results if r is not None]
results
len(results)

In [ ]:
results = results[:10]

In [ ]:
original_results_df = pd.DataFrame(results)
original_results_df

In [ ]:
import plotly.express as px

new_results = []
for r in sorted(results, key=lambda x: x['imbalance'], reverse=False):
    new_line1 = deepcopy(r)
    new_line2 = deepcopy(r)

    new_line1['desimbalancer'] = False
    new_line1['result'] = new_line1['result_false']
    del new_line1['result_false']
    del new_line1['result_true']

    new_line2['desimbalancer'] = True
    new_line2['result'] = new_line2['result_true']
    del new_line2['result_false']
    del new_line2['result_true']

    new_results.append(new_line1)
    new_results.append(new_line2)

results_df = pd.DataFrame(new_results)
results_df['id'] = results_df['id'].astype(str)

fig = px.bar(results_df, x='id', y='result', color='desimbalancer', barmode='group',
    #    title='Quantidade de memória utilizada por tabuleiro',
    #    labels={
    #         'game_id': 'Identificador do tabuleiro',
    #         'memory_used': 'Quantidade de blocos de memória utilizada',
    #         'alg': 'Algoritmo'
    #    }
    custom_data=['result', 'imbalance', 'n_rows', 'n_cols']
)

fig.update_traces(
    hovertemplate="<br>".join([
        "Evaluation: %{customdata[0]:.4f}",
        "Imbalance: %{customdata[1]:.4f}",
        "N Rows: %{customdata[2]}",
        "N Cols: %{customdata[3]}",
    ])
)

## Amount of labels

In [ ]:
n_labels_dict = dict()

for ds in datasets:
    df = ds['X']
    id = ds['id']
    last_col = df.columns[-1]
    n_labels_dict[id] = len(df[last_col].unique())

In [ ]:
print('(id, n_labels)')
sorted(n_labels_dict.items(), key=lambda x: x[1], reverse=True)